# Logistic Regression

## Basics

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('mylogreg').getOrCreate()

from pyspark.ml.classification import LogisticRegression

my_data = spark.read.format('libsvm').load('/FileStore/tables/sample_libsvm_data.txt')

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('mylogreg').getOrCreate()

from pyspark.ml.classification import LogisticRegression

my_data = spark.read.format('libsvm').load('/FileStore/tables/sample_libsvm_data.txt')

In [0]:
my_log_reg_model = LogisticRegression()
fitted_logreg = my_log_reg_model.fit(my_data)

In [0]:
log_summary = fitted_logreg.summary

In [0]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [0]:
lr_train, lr_test = my_data.randomSplit([0.7,0.3])
final_model = LogisticRegression()
fit_final = final_model.fit(lr_train)

In [0]:
preds_and_labels = fit_final.evaluate(lr_test)
preds_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[0.50695378662002...|[0.62409209989795...|       0.0|
|  0.0|(692,[122,123,148...|[22.6728179516727...|[0.99999999985766...|       0.0|
|  0.0|(692,[123,124,125...|[20.4360505335402...|[0.99999999866728...|       0.0|
|  0.0|(692,[124,125,126...|[90.2752649803173...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[28.7753894624751...|[0.99999999999968...|       0.0|
|  0.0|(692,[126,127,128...|[27.1966089516400...|[0.99999999999845...|       0.0|
|  0.0|(692,[129,130,131...|[19.0705541124891...|[0.99999999477888...|       0.0|
|  0.0|(692,[153,154,155...|[30.2362391899831...|[0.99999999999992...|       0.0|
|  0.0|(692,[153,154,155...|[28.3853452304096...|[0.99999999999952...|       0.0|
|  0.0|(692,[154

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [0]:
my_eval = BinaryClassificationEvaluator()
final_roc = my_eval.evaluate(preds_and_labels.predictions)
final_roc

Out[18]: 1.0

## Log Reg code along
Using StrinIndexing, OneHotEncoder and Pipeline

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('mylogreg2').getOrCreate()

from pyspark.ml.classification import LogisticRegression

df = spark.sql("SELECT * FROM titanic_csv")

In [0]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [0]:
my_cols = df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])
my_final_data = my_cols.na.drop()

In [0]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,
                               OneHotEncoder, StringIndexer)

In [0]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [0]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkedIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkedIndex', outputCol='EmbarkVec')

In [0]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],
                           outputCol='features')

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [0]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [0]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer,
                           gender_encoder, embark_encoder,
                           assembler, log_reg_titanic])

In [0]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

In [0]:
fit_model = pipeline.fit(train_data)

In [0]:
results = fit_model.transform(test_data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [0]:
AUC = my_eval.evaluate(results)
AUC

Out[24]: 0.7888326614632473